# 9-5. 콘텐츠 기반 필터링 실습 - TMDB 5000 영화 데이터 세트

- 콘텐츠 기반 필터링이란? 사용자가 특정 영화를 감상하고 그 영화를 좋아했다면 그 영화와 비슷한 특성/속성, 구성 요소 등을 가진 다른 영화를 추천하는 것 >> 영화/상품/서비스 간의 유사성을 판단하는 기준이 이를 구성하는 다양한 컨텐츠(장르, 감독, 배우, 평점, 키워드, 영화 설명)를 기반으로 하는 방식
- 여기서는 영화 장르 속성을 기반으로 ㄱㄱ
- 장르 칼럼 값의 유사도를 비교하고 높은 평점의 영화 추천하기

### 1. 데이터 로딩 및 가공

In [3]:
import pandas as pd
import numpy as np
import warnings; warnings.filterwarnings('ignore')

movies=pd.read_csv('/Users/bluecloud/Documents/대학/유런/데이터셋/tmdb_5000/tmdb_5000_movies.csv')
print(movies.shape)
movies.head(3)

(4803, 20)


,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466


>> 1. 콘텐츠 기반 필터링 추천 분석에 사용할 주요 칼럼만 추출하기
>> ex) id, title, genres, vote_average, vote_count, popularity, keywords, overview
>> 2-1. genres, keywords의 딕셔너리 형태 확인해보기

In [7]:
movies_df=movies[['id', 'title', 'genres', 'vote_average', 'vote_count', 'popularity', 'keywords', 'overview']]

In [13]:
pd.set_option('max_colwidth',100)
movies_df[['genres','keywords']][:1]

,genres,keywords
0,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""name"": ""Fantasy""}, {...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"": 2964, ""name"": ""future""}, {""id"": 3386, ""name"": ""sp..."


>> 2-2. 딕셔너리 키 name으로 추출하기
>> 개별 장르, 키워드를 파이썬 객체로 추출하기 literal_eval()

In [15]:
from ast import literal_eval
movies_df['genres']=movies_df['genres'].apply(literal_eval)
movies_df['keywords']=movies_df['keywords'].apply(literal_eval)

In [17]:
# 리스트 내 여러 개의 딕셔너리의 'name'키에 해당하는 값을 찾아 리스트 객체로 변환하기
movies_df['genres']=movies_df['genres'].apply(lambda x:[y['name'] for y in x])
movies_df['keywords']=movies_df['keywords'].apply(lambda x:[y['name'] for y in x])
movies_df[['genres','keywords']][:1]

,genres,keywords
0,"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colony, society, space travel, futuristic, romance, spa..."


### 2. 장르 콘텐츠 유사도 측정

- 영화 A와 B의 장르가 여러개일 때 어떻게 유사도 측정?
- (1) genres를 문자열로 변경 (2) CountVectorizer로 피처 벡터화하기 (3) 행렬 데이터 값을 코사인 유사도로 비교
- !! 리스트 객체 값으로 구성된 genres 칼럼을 apply(lamnda x:('').join(리스트 객체))를 적용해 공백 문자로 구분하는 문자열로 변환하기

In [25]:
from sklearn.feature_extraction.text import CountVectorizer

# CountVectorizer를 적용하기 위해 공백문자로 word 단위가 구분되는 문자열로 변환
movies_df['genres_literal'] = movies_df['genres'].apply(lambda x : (' ').join(x))
count_vect = CountVectorizer(min_df=0.0, ngram_range=(1,2))
genre_mat = count_vect.fit_transform(movies_df['genres_literal'])
print(genre_mat.shape)

(4803, 276)


- 사이킷런의 cosine_similarity() : 기준 행과 비교 대상 행간의 코사인 유사도를 행렬 형태로 반환

In [27]:
from sklearn.metrics.pairwise import cosine_similarity

genre_sim=cosine_similarity(genre_mat,genre_mat)
print(genre_sim.shape)
print(genre_sim[:1])

(4803, 4803)
[[1.         0.59628479 0.4472136  ... 0.         0.         0.        ]]


>> cosine_similarity() 호출로 생성된 genre_sim 객체는 movies_df의 genre_literal 칼럼을 피처 벡터화한 행렬(genre_mat) 데이터의 행(레코드)별 유사도 정보를 가짐
>>  movies_df DataFrame의 행별 장르 유사도 값을 의미
>> 장르 기준으로 콘텐츠 기반 필터링 수행시 개별 record에 대해 장르 유사도가 높은 순으로 다른 레코드 추출할 때 genre_sim 객체를 이용

- argsort()[:,::-1]는 유사도가 높은 순으로 정리가 되는데..추출되는건 유사도 값이 아닌 비교 대상의 위치 인덱스 값을 가져옴

In [29]:
genre_sim_sorted_ind=genre_sim.argsort()[:,::-1]
print(genre_sim_sorted_ind[:1])

[[   0 3494  813 ... 3038 3037 2401]]


>> 위의 숫자들은 n번 레코드를 의미

### 3. 장르 콘텐츠 필터링을 이용한 영화 추천

- 장르 유사도에 따라 영화 추천하는 함수
- 인자로 기반데이터 movies_df, 장르 코사인 유사도 인덱스 genre_sim_sorted_ind, 영화 제목, 추천 영화 정보 df

In [31]:
def find_sim_movie(df,sorted_ind,title_name,top_n=10):
    #인자로 입력된 movies_df에서 'title' 칼럼이 입력된 title_name값인 DataFrame 추출
    title_movie=df[df['title']==title_name]
    #title_named을 가진 df의 index 객체를 ndarray로 반환하고 sorted_ind 인자로 입력된 genre_sim_sorted_ind 객체에서 유사도 순으로 top_n개의 index 추출
    title_index=title_movie.index.values
    similar_indexes=sorted_ind[title_index,:(top_n)]
    # 추출된 top_n index 출력, top_n index는 2차원 데이터
    #dataframe에서 index로 사용하기 위해 1차원 array로 변경
    print(similar_indexes)
    similar_indexes=similar_indexes.reshape(-1)
    return df.iloc[similar_indexes]


In [33]:
# 영화 '대부'와 장르별 유사한 영화 10개 추천하기
similar_movies=find_sim_movie(movies_df,genre_sim_sorted_ind,'The Godfather',10)
similar_movies[['title', 'vote_average']]

[[1370 4041 3337 1847 3378 4217 2839  281  588 3866]]


,title,vote_average
1370,21,6.5
4041,This Is England,7.4
3337,The Godfather,8.4
1847,GoodFellas,8.2
3378,Auto Focus,6.1
4217,Kids,6.8
2839,Rounders,6.9
281,American Gangster,7.4
588,Wall Street: Money Never Sleeps,5.8
3866,City of God,8.1


- 영화 평점 순으로 세워 살펴보기

In [35]:
movies_df[['title', 'vote_average','vote_count']].sort_values('vote_average',ascending=False)[:10]

,title,vote_average,vote_count
3519,Stiff Upper Lips,10.0,1
4247,Me You and Five Bucks,10.0,2
4045,"Dancer, Texas Pop. 81",10.0,1
4662,Little Big Top,10.0,1
3992,Sardaarji,9.5,2
2386,One Man's Hero,9.3,2
2970,There Goes My Baby,8.5,2
1881,The Shawshank Redemption,8.5,8205
2796,The Prisoner of Zenda,8.4,11
3337,The Godfather,8.4,5893


>> 평가 횟수가 적은 영화들이 상위에 랭크되어있음 >> 왜곡된 평점 데이터
>> 평가 횟수에 대한 가중치가 부여된 평점 방식 사용
>> V : vote_count / R : vote_average / C : 전체 영화의 평균 평점
>> m 값이 높아지면 평점 투표 횟수에 더 많은 가중치가 부여됨

In [37]:
C=movies_df['vote_average'].mean()
# 전체 투표에서 상위 60%에 해당하는 횟수를 기준으로 투표 횟수에 대한 가중치 정하기
m=movies_df['vote_count'].quantile(0.6)
print('C:',round(C,3),'m:',round(m,3))

C: 6.092 m: 370.2


- 기존 평점을 새로운 가중 평점으로 변경하는 함수생성 >> df의 레코드를 인자로 받아 레코드별 가중 평점 반환
- 새로운 평점 정보 만들기

In [47]:
percentile=0.6
m=movies_df['vote_count'].quantile(percentile)
C=movies_df['vote_average'].mean()

def weighted_vote_average(record):
    v=record['vote_count']
    R=record['vote_average']
    return ((v/(v+m))*R)+((m/(v+m))*C)
    
movies_df['weighted_vote']=movies_df.apply(weighted_vote_average,axis=1)

In [49]:
movies_df[['title','vote_average','weighted_vote','vote_count']].sort_values('weighted_vote',ascending=False)[:10]

,title,vote_average,weighted_vote,vote_count
1881,The Shawshank Redemption,8.5,8.396052,8205
3337,The Godfather,8.4,8.263591,5893
662,Fight Club,8.3,8.216455,9413
3232,Pulp Fiction,8.3,8.207102,8428
65,The Dark Knight,8.2,8.136930,12002
1818,Schindler's List,8.3,8.126069,4329
3865,Whiplash,8.3,8.123248,4254
809,Forrest Gump,8.2,8.105954,7927
2294,Spirited Away,8.3,8.105867,3840
2731,The Godfather: Part II,8.3,8.079586,3338


- 장르 유사성이 높은 영화를 top_n의 2배수만큼 후보군으로 선정하고 weighted_vote 칼럼 값이 높은 순으로 top_n만큼 추출하는 함수

In [55]:
def find_sim_movie(df,sorted_ind,title_name,top_n=10):
    title_movie=df[df['title']==title_name]
    title_index=title_movie.index.values
    #top_n의 2배수만큼 후보군으로 선정
    similar_indexes=sorted_ind[title_index,:(top_n*2)]
    similar_indexes=similar_indexes.reshape(-1)
    #기준 영화 인덱스 제외
    similar_indexes=similar_indexes[similar_indexes!=title_index]
    #top_n의 2배에 해당하는 후보군에서 weighted_vote가 높은 순으로 top_n만큼 추출
    return df.iloc[similar_indexes].sort_values('weighted_vote',ascending=False)[:top_n]
similar_movies=find_sim_movie(movies_df,genre_sim_sorted_ind,'The Godfather',10)
similar_movies[['title', 'vote_average','weighted_vote']]

,title,vote_average,weighted_vote
1881,The Shawshank Redemption,8.5,8.396052
2731,The Godfather: Part II,8.3,8.079586
1847,GoodFellas,8.2,7.976937
3866,City of God,8.1,7.759693
1663,Once Upon a Time in America,8.2,7.657811
892,Casino,7.8,7.423040
281,American Gangster,7.4,7.141396
4041,This Is England,7.4,6.739664
1149,American Hustle,6.8,6.717525
1243,Mean Streets,7.2,6.626569


# 9-6. 아이템 기반 최근접 이웃 협업 필터링 실습

- 추천 정확도가 더 뛰어난 아이템 기반의 협업 필터링
- Grouplens 사이트에서 만든 MovieLens 데이터 세트

### 1. 데이터 가공 및 변환

In [1]:
import pandas as pd
import numpy as np

movies=pd.read_csv('/Users/bluecloud/Documents/대학/유런/데이터셋/ml-latest-small/movies.csv')
ratings=pd.read_csv('/Users/bluecloud/Documents/대학/유런/데이터셋/ml-latest-small/ratings.csv')
print(movies.shape)
print(ratings.shape)

(9742, 3)
(100836, 4)


In [5]:
movies.head(3)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance


In [7]:
ratings.head(3)

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224


movies.csv : 영화에 대한 title과 genres를 가지고 있음
ratings.csv : 사용자별로 영화에 대한 평점을 매김
>> 사용자 로우 - 영화 평점 칼럼 데이터로 변환 필요 : pivot_table('테이블값',index='',columns='') 함수

In [9]:
ratings=ratings[['userId','movieId','rating']]
ratings_matrix=ratings.pivot_table('rating',index='userId',columns='movieId')
ratings_matrix.head(3)

movieId,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,34,36,38,39,40,41,42,43,...,185135,185435,185473,185585,186587,187031,187541,187593,187595,187717,188189,188301,188675,188751,188797,188833,189043,189111,189333,189381,189547,189713,190183,190207,190209,190213,190215,190219,190221,191005,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,4.0,NaN,4.0,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


- movieId는 가독성이 떨어지므로 movies.csv에서 movieId>>title로 변경 & pivot_table()에서 columns을 title로
- NaN은 0으로

In [15]:
#title 칼럼을 얻기 위해 movies
ratings_movies=pd.merge(ratings,movies,on='movieId')
#columns을 title로
ratings_matrix=ratings_movies.pivot_table('rating',index='userId',columns='title')
#NaN 값을 모두 0으로 변환
ratings_matrix=ratings_matrix.fillna(0)
ratings_matrix.head(3)

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...All the Marbles (1981),...And Justice for All (1979),00 Schneider - Jagd auf Nihil Baxter (1994),1-900 (06) (1994),10 (1979),10 Cent Pistol (2015),10 Cloverfield Lane (2016),10 Items or Less (2006),10 Things I Hate About You (1999),10 Years (2011),"10,000 BC (2008)",100 Girls (2000),100 Streets (2016),101 Dalmatians (1996),101 Dalmatians (One Hundred and One Dalmatians) (1961),101 Dalmatians II: Patch's London Adventure (2003),101 Reykjavik (101 Reykjavík) (2000),102 Dalmatians (2000),10th & Wolf (2006),"10th Kingdom, The (2000)","10th Victim, The (La decima vittima) (1965)","11'09""01 - September 11 (2002)",11:14 (2003),"11th Hour, The (2007)",12 Angry Men (1957),12 Angry Men (1997),12 Chairs (1971),12 Chairs (1976),12 Rounds (2009),12 Years a Slave (2013),...,Zathura (2005),Zatoichi and the Chest of Gold (Zatôichi senryô-kubi) (Zatôichi 6) (1964),Zazie dans le métro (1960),Zebraman (2004),"Zed & Two Noughts, A (1985)",Zeitgeist: Addendum (2008),Zeitgeist: Moving Forward (2011),Zeitgeist: The Movie (2007),Zelary (2003),Zelig (1983),Zero Dark Thirty (2012),Zero Effect (1998),"Zero Theorem, The (2013)",Zero de conduite (Zero for Conduct) (Zéro de conduite: Jeunes diables au collège) (1933),Zeus and Roxanne (1997),Zipper (2015),Zodiac (2007),Zombeavers (2014),Zombie (a.k.a. Zombie 2: The Dead Are Among Us) (Zombi 2) (1979),Zombie Strippers! (2008),Zombieland (2009),Zone 39 (1997),"Zone, The (La Zona) (2007)",Zookeeper (2011),Zoolander (2001),Zoolander 2 (2016),Zoom (2006),Zoom (2015),Zootopia (2016),Zulu (1964),Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### 2. 영화 간 유사도 산출

- ratings_matrix를 이용해 영화 간 유사도 측정 >> cosine_similarity()로 유사도 산출 but 위의 사례는 사용자 간의 유사도가 산출됨
  >> 행과 열을 반대로 transpose()

In [17]:
from sklearn.metrics.pairwise import cosine_similarity

ratings_matrix_T=ratings_matrix.transpose()
ratings_matrix_T.head(3)

userId,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,571,572,573,574,575,576,577,578,579,580,581,582,583,584,585,586,587,588,589,590,591,592,593,594,595,596,597,598,599,600,601,602,603,604,605,606,607,608,609,610
title,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
'71 (2014),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
'Hellboy': The Seeds of Creation (2004),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
'Round Midnight (1986),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [19]:
# 영화 간 유사도 산출
item_sim=cosine_similarity(ratings_matrix_T,ratings_matrix_T)

#cosine_similarity()로 반환된 넘파이 행렬을 영화명으로 매핑해 df로 변환
item_sim_df=pd.DataFrame(data=item_sim,index=ratings_matrix.columns,columns=ratings_matrix.columns)
print(item_sim_df.shape)
item_sim_df.head(3)

(9719, 9719)


title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...All the Marbles (1981),...And Justice for All (1979),00 Schneider - Jagd auf Nihil Baxter (1994),1-900 (06) (1994),10 (1979),10 Cent Pistol (2015),10 Cloverfield Lane (2016),10 Items or Less (2006),10 Things I Hate About You (1999),10 Years (2011),"10,000 BC (2008)",100 Girls (2000),100 Streets (2016),101 Dalmatians (1996),101 Dalmatians (One Hundred and One Dalmatians) (1961),101 Dalmatians II: Patch's London Adventure (2003),101 Reykjavik (101 Reykjavík) (2000),102 Dalmatians (2000),10th & Wolf (2006),"10th Kingdom, The (2000)","10th Victim, The (La decima vittima) (1965)","11'09""01 - September 11 (2002)",11:14 (2003),"11th Hour, The (2007)",12 Angry Men (1957),12 Angry Men (1997),12 Chairs (1971),12 Chairs (1976),12 Rounds (2009),12 Years a Slave (2013),...,Zathura (2005),Zatoichi and the Chest of Gold (Zatôichi senryô-kubi) (Zatôichi 6) (1964),Zazie dans le métro (1960),Zebraman (2004),"Zed & Two Noughts, A (1985)",Zeitgeist: Addendum (2008),Zeitgeist: Moving Forward (2011),Zeitgeist: The Movie (2007),Zelary (2003),Zelig (1983),Zero Dark Thirty (2012),Zero Effect (1998),"Zero Theorem, The (2013)",Zero de conduite (Zero for Conduct) (Zéro de conduite: Jeunes diables au collège) (1933),Zeus and Roxanne (1997),Zipper (2015),Zodiac (2007),Zombeavers (2014),Zombie (a.k.a. Zombie 2: The Dead Are Among Us) (Zombi 2) (1979),Zombie Strippers! (2008),Zombieland (2009),Zone 39 (1997),"Zone, The (La Zona) (2007)",Zookeeper (2011),Zoolander (2001),Zoolander 2 (2016),Zoom (2006),Zoom (2015),Zootopia (2016),Zulu (1964),Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
title,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
'71 (2014),1.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.141653,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.285169,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.65561,0.0,0.0,0.0,0.212814,0.919145,0.0,0.0,0.120996,0.0,0.0,0.0,0.149201,0.0,0.0,0.0,0.178042,0.0,0.0,0.342055,0.543305,0.707107,0.0,0.0,0.139431,0.327327,0.0,0.0
'Hellboy': The Seeds of Creation (2004),0.0,1.000000,0.707107,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.715542,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.150269,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.124109,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.148970,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0
'Round Midnight (1986),0.0,0.707107,1.000000,0.0,0.0,0.0,0.176777,0.0,0.000000,0.0,0.707107,0.505964,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.106256,0.0,0.0,0.0,0.0,0.0,0.707107,0.0,0.0,0.0,0.197457,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.105338,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0


>> 영화의 유사도 행렬

In [21]:
# GodFather과 유사한 영화 6개 추출하기
item_sim_df["Godfather, The (1972)"].sort_values(ascending=False)[:6]

title
Godfather, The (1972)                        1.000000
Godfather: Part II, The (1974)               0.821773
Goodfellas (1990)                            0.664841
One Flew Over the Cuckoo's Nest (1975)       0.620536
Star Wars: Episode IV - A New Hope (1977)    0.595317
Fargo (1996)                                 0.588614
Name: Godfather, The (1972), dtype: float64

In [23]:
# Inception과 유사한 영화 추출하기(자기 자신 제외)
item_sim_df["Inception (2010)"].sort_values(ascending=False)[1:6]

title
Dark Knight, The (2008)          0.727263
Inglourious Basterds (2009)      0.646103
Shutter Island (2010)            0.617736
Dark Knight Rises, The (2012)    0.617504
Fight Club (1999)                0.615417
Name: Inception (2010), dtype: float64

### 3. 아이템 기반 최근접 이웃 협업 필터링으로 개인화된 영화 추천

- 영화 유사도 데이터 + 개인에게 최적화된 영화 추천 구현
- !아직 관람하지 않은 영화 추천
- 예측 평점 식은 교재에서 확인

- 함수 predict_rating() : 영화 간 유사도 df인 item_sim_df와 사용자-영화 평점 df ratings_matrix 변수를 활용하여 사용자별 최적화된 평점 스코어를 예측하는 함수 >> 둘다 넘파이 행렬로 변환
- 모든 영화와의 코사인 유사도를 벡터 내적 곱(dot)하여 정규화를 위해 유사도 벡터 합으로 나눈다

In [25]:
def predict_rating(ratings_arr,item_sim_arr):
    ratings_pred=ratings_arr.dot(item_sim_arr)/np.array([np.abs(item_sim_arr).sum(axis=1)])
    return ratings_pred

In [27]:
ratings_pred=predict_rating(ratings_matrix.values,item_sim_df.values)
ratings_pred_matrix=pd.DataFrame(data=ratings_pred,index=ratings_matrix.index,columns=ratings_matrix.columns)
ratings_pred_matrix.head(3)

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...All the Marbles (1981),...And Justice for All (1979),00 Schneider - Jagd auf Nihil Baxter (1994),1-900 (06) (1994),10 (1979),10 Cent Pistol (2015),10 Cloverfield Lane (2016),10 Items or Less (2006),10 Things I Hate About You (1999),10 Years (2011),"10,000 BC (2008)",100 Girls (2000),100 Streets (2016),101 Dalmatians (1996),101 Dalmatians (One Hundred and One Dalmatians) (1961),101 Dalmatians II: Patch's London Adventure (2003),101 Reykjavik (101 Reykjavík) (2000),102 Dalmatians (2000),10th & Wolf (2006),"10th Kingdom, The (2000)","10th Victim, The (La decima vittima) (1965)","11'09""01 - September 11 (2002)",11:14 (2003),"11th Hour, The (2007)",12 Angry Men (1957),12 Angry Men (1997),12 Chairs (1971),12 Chairs (1976),12 Rounds (2009),12 Years a Slave (2013),...,Zathura (2005),Zatoichi and the Chest of Gold (Zatôichi senryô-kubi) (Zatôichi 6) (1964),Zazie dans le métro (1960),Zebraman (2004),"Zed & Two Noughts, A (1985)",Zeitgeist: Addendum (2008),Zeitgeist: Moving Forward (2011),Zeitgeist: The Movie (2007),Zelary (2003),Zelig (1983),Zero Dark Thirty (2012),Zero Effect (1998),"Zero Theorem, The (2013)",Zero de conduite (Zero for Conduct) (Zéro de conduite: Jeunes diables au collège) (1933),Zeus and Roxanne (1997),Zipper (2015),Zodiac (2007),Zombeavers (2014),Zombie (a.k.a. Zombie 2: The Dead Are Among Us) (Zombi 2) (1979),Zombie Strippers! (2008),Zombieland (2009),Zone 39 (1997),"Zone, The (La Zona) (2007)",Zookeeper (2011),Zoolander (2001),Zoolander 2 (2016),Zoom (2006),Zoom (2015),Zootopia (2016),Zulu (1964),Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,0.070345,0.577855,0.321696,0.227055,0.206958,0.194615,0.249883,0.102542,0.157084,0.178197,0.119402,0.185026,0.269199,0.521031,0.141683,0.116623,0.135441,0.224885,0.226528,0.113608,0.185277,0.303638,0.113608,0.255040,0.260446,0.326968,0.305769,0.155031,0.348717,0.186870,0.119402,0.099756,0.206331,0.348717,0.267407,0.237128,0.050947,0.050947,0.200747,0.156893,...,0.186554,0.050947,0.040443,0.121184,0.178482,0.104488,0.104488,0.110808,0.102542,0.175859,0.179162,0.231606,0.093467,0.094357,0.112690,0.113608,0.164231,0.086360,0.277215,0.262709,0.180320,0.112690,0.111653,0.130131,0.248312,0.132009,0.285913,0.113608,0.155861,0.155927,0.113608,0.181738,0.133962,0.128574,0.006179,0.212070,0.192921,0.136024,0.292955,0.720347
2,0.018260,0.042744,0.018861,0.000000,0.000000,0.035995,0.013413,0.002314,0.032213,0.014863,0.000000,0.005220,0.093722,0.000000,0.014296,0.016398,0.043685,0.019004,0.020071,0.015640,0.028349,0.043477,0.015640,0.019634,0.016893,0.008251,0.010919,0.013711,0.000000,0.020300,0.000000,0.002726,0.022639,0.000000,0.032268,0.031130,0.040699,0.040699,0.024950,0.043495,...,0.021269,0.040699,0.030610,0.019721,0.002215,0.023352,0.023352,0.028403,0.002314,0.006791,0.033143,0.010933,0.018806,0.003525,0.011425,0.015640,0.030904,0.017290,0.019250,0.039449,0.038895,0.011425,0.035400,0.038101,0.034181,0.026764,0.000000,0.015640,0.037980,0.006859,0.015640,0.020855,0.020119,0.015745,0.049983,0.014876,0.021616,0.024528,0.017563,0.000000
3,0.011884,0.030279,0.064437,0.003762,0.003749,0.002722,0.014625,0.002085,0.005666,0.006272,0.091413,0.007483,0.018710,0.080626,0.006995,0.006766,0.006988,0.005427,0.006743,0.006923,0.005389,0.008943,0.006923,0.008559,0.009333,0.006316,0.031652,0.007376,0.009832,0.022056,0.091413,0.002548,0.008762,0.009832,0.008773,0.004379,0.001117,0.001117,0.007007,0.005163,...,0.008810,0.001117,0.000000,0.010758,0.007362,0.003726,0.0

>> 예측 평점이 사용자별 영화의 실제 평점과 코사인 유사도 내적한 값이어서 0에 해당했던 실제 영화 평점이 예측에서는 값이 부여되는 경우가 많아 실제 평점에 비해 작을 수 있다.

- 위의 예측 결과가 원래의 실 평저모가 얼마나 차이가 나는지 확인해보기 : MSE지표
- 실제와 예측 평점의 차이는 기존에 평점이 부여된 데이터에 대해서만 오차 정도 측정

In [31]:
from sklearn.metrics import mean_squared_error

# 사용자가 평점을 부여한 영화에 대해 MSE 구하기
def get_mse(pred,actual):
    #평점이 있는 실제 영화만 추출
    pred=pred[actual.nonzero()].flatten()
    actual=actual[actual.nonzero()].flatten()
    return mean_squared_error(pred,actual)
print('아이템 기반 모든 최근접 이웃 MSE : ',get_mse(ratings_pred,ratings_matrix.values))

아이템 기반 모든 최근접 이웃 MSE :  9.895354759094706


>> mse를 감소시키는 방향으로 개선하기

- 기존의 predict_rating() : 해당 영화와 다른 모든 영화 간의 유사도 벡터 적용 >> 상대적 평점 예측 떨어짐
- 새로운 predict_rating_topsim() : 특정 영화와 비슷한 유사도를 갖는 영화(N값으로 개수 정하기)에 대하여 유사도 벡터 적용하기

In [47]:
def predict_rating_topsim(ratings_arr,item_sim_arr,n=20):
    #사용자-아이템 평점 행렬 크기만큼 0으로 채운 예측 행렬 초기화
    pred=np.zeros(ratings_arr.shape)
    #사용자-아이템 평점 행렬 열 크기만큼 루프 수행
    for col in range(ratings_arr.shape[1]):
        #유사도 행렬에서 유사도 큰 순 n개 데이터 행렬의 인덱스 변환
        top_n_items=[np.argsort(item_sim_arr[:,col])[:-n-1:-1]]
        #개인화된 예측 평점을 계산
        for row in range(ratings_arr.shape[0]):
            pred[row, col] = item_sim_arr[col, :][top_n_items].dot(ratings_arr[row, :][top_n_items].T) 
            pred[row, col] /= np.sum(np.abs(item_sim_arr[col, :][top_n_items]))        
    return pred

- 위의 함수로 예측 평점을 계산하고 실제 평점과 MSE 구해보기

In [49]:
ratings_pred = predict_rating_topsim(ratings_matrix.values , item_sim_df.values, n=20)
print('아이템 기반 인접 TOP-20 이웃 MSE: ', get_mse(ratings_pred, ratings_matrix.values ))

# 계산된 예측 평점 데이터는 DataFrame으로 재생성
ratings_pred_matrix = pd.DataFrame(data=ratings_pred, index= ratings_matrix.index,columns = ratings_matrix.columns)

/var/folders/2c/j3wmswps2r5gclvj1skrczqh0000gn/T/ipykernel_32545/4158240769.py:10: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  pred[row, col] = item_sim_arr[col, :][top_n_items].dot(ratings_arr[row, :][top_n_items].T)


아이템 기반 인접 TOP-20 이웃 MSE:  3.6949999176225483


- MSE가 많이 향상됨
- 특정 사용자에 대한 영화 추천 : userId 기준으로 입력

In [51]:
# userId=9
user_rating_id=ratings_matrix.loc[9,:]
user_rating_id[user_rating_id>0].sort_values(ascending=False)[:10]

title
Adaptation (2002)                                                                 5.0
Citizen Kane (1941)                                                               5.0
Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981)    5.0
Producers, The (1968)                                                             5.0
Lord of the Rings: The Two Towers, The (2002)                                     5.0
Lord of the Rings: The Fellowship of the Ring, The (2001)                         5.0
Back to the Future (1985)                                                         5.0
Austin Powers in Goldmember (2002)                                                5.0
Minority Report (2002)                                                            4.0
Witness (1985)                                                                    4.0
Name: 9, dtype: float64

- userId=9한테 아이템 기반 협업 필터링을 통해 영화 추천하기 >> 평점 주지 않은 영화 리스트로 생성

In [53]:
def get_unseen_movies(ratings_matrix,userId):
    #userId로 입력받은 사용자의 모든 영화 정보를 추출해 Series로 반환
    #반환된 user_rating으 title을 인덱스로 가짐
    user_rating=ratings_matrix.loc[userId,:]
    
    #user_rating이 0보다 크다? >> 기존에 관람한 영화
    already_seen=user_rating[user_rating>0].index.tolist()
    
    #모든 영화명을 list 객체로 만듦
    movies_list=ratings_matrix.columns.tolist()
    
    #already_seen을 제외하고 movie_list에서 제외
    unseen_list=[movie for movie in movies_list if movie not in already_seen]
    
    return unseen_list


In [55]:
def recomm_movie_by_userid(pred_df, userId, unseen_list, top_n=10):
    # 예측 평점 DataFrame에서 사용자id index와 unseen_list로 들어온 영화명 컬럼을 추출하여
    # 가장 예측 평점이 높은 순으로 정렬함. 
    recomm_movies = pred_df.loc[userId, unseen_list].sort_values(ascending=False)[:top_n]
    return recomm_movies
    
# 사용자가 관람하지 않는 영화명 추출   
unseen_list = get_unseen_movies(ratings_matrix, 9)

# 아이템 기반의 인접 이웃 협업 필터링으로 영화 추천 
recomm_movies = recomm_movie_by_userid(ratings_pred_matrix, 9, unseen_list, top_n=10)

# 평점 데이타를 DataFrame으로 생성. 
recomm_movies = pd.DataFrame(data=recomm_movies.values,index=recomm_movies.index,columns=['pred_score'])
recomm_movies

,pred_score
title,
Shrek (2001),0.866202
Spider-Man (2002),0.857854
"Last Samurai, The (2003)",0.817473
Indiana Jones and the Temple of Doom (1984),0.816626
"Matrix Reloaded, The (2003)",0.800990
Harry Potter and the Sorcerer's Stone (a.k.a. Harry Potter and the Philosopher's Stone) (2001),0.765159
Gladiator (2000),0.740956
"Matrix, The (1999)",0.732693
Pirates of the Caribbean: The Curse of the Black Pearl (2003),0.689591


# 9-7. 행렬 분해를 이용한 잠재 요인 협업 필터링 실습

- 행렬 분해 잠재 요인 협업 필터링 : 주로 SVD나 NMF 등이 적용됨 >> Null 데이터가 많아 SGD나 ALS 기반의 행렬 분해 활용됨
- P와 Q 행렬로 계산된 예측 R 행렬 값이 실제 R 행렬 값과 최소한의 오류를 가질 수 있도록 반복적으로 비용함수를 최적화함으로써 적합한 P와 Q 행렬을 유추
- 행렬 분해 로직을 새로운 함수로 정리 : R은 원본 사용자-아이템 평점 행렬, K는 잠재요인의 차원 수, steps는 SGD의 반복 횟수, learning_rate는 학습률, r_lambda는 L2규제 계수

In [57]:
from sklearn.metrics import mean_squared_error
 
def get_rmse(R, P, Q, non_zeros):
    error = 0
    # 두개의 분해된 행렬 P와 Q.T의 내적으로 예측 R 행렬 생성
    full_pred_matrix = np.dot(P, Q.T)
     
    # 실제 R 행렬에서 널이 아닌 값의 위치 인덱스 추출하여 실제 R 행렬과 예측 행렬의 RMSE 추출
    x_non_zero_ind = [non_zero[0] for non_zero in non_zeros]
    y_non_zero_ind = [non_zero[1] for non_zero in non_zeros]
    R_non_zeros = R[x_non_zero_ind, y_non_zero_ind]
    full_pred_matrix_non_zeros = full_pred_matrix[x_non_zero_ind, y_non_zero_ind]
       
    mse = mean_squared_error(R_non_zeros, full_pred_matrix_non_zeros)
    rmse = np.sqrt(mse)
     
    return rmse

In [69]:
def matrix_factorization(R,K,steps=200,learning_rate=0.01,r_lambda=0.01):
    num_users,num_items=R.shape
    #P와 Q 매트릭스의 크기를 지정하고 정규 분포를 가진 랜덤한 값으로 입력
    np.random.seed(1)
    P=np.random.normal(scale=1./K,size=(num_users,K))
    Q=np.random.normal(scale=1./K,size=(num_items,K))
    
    prev_rmse=10000
    break_count=0
    
    #R>0인 행, 열 위치, 값을 non_zero 리스트 객체에 저장
    non_zeros=[(i,j,R[i,j]) for i in range(num_users) for j in range(num_items) if R[i,j]>0]
    
    #SGD 기법으로 PQ matrix를 업데이트
    for step in range(steps):
        for i,j,r in non_zeros:
            #실제값과 예측값 차이인 오류값 구하기
            eij=r-np.dot(P[i,:],Q[j,:].T)
            #정규화를 반영한 SGD 업데이트 공식 적용
            P[i,:]=P[i,:]+learning_rate*(eij*Q[j,:]-r_lambda*P[i,:])
            Q[j,:]=Q[j,:]+learning_rate*(eij*P[i,:]-r_lambda*Q[j,:])
        rmse=get_rmse(R, P, Q, non_zeros)
        if(step%10)==0:
            print("### iteration step :", step, " rmse: ", rmse)
    return P,Q  

- 영화 평점 데이터를 새롭게 df로 로딩하고 사용자-아이템 평점 행렬로 만들기

In [65]:
import pandas as pd
import numpy as np

movies=pd.read_csv('/Users/bluecloud/Documents/대학/유런/데이터셋/ml-latest-small/movies.csv')
ratings=pd.read_csv('/Users/bluecloud/Documents/대학/유런/데이터셋/ml-latest-small/ratings.csv')
ratings=ratings[['userId','movieId','rating']]
ratings_matrix=ratings.pivot_table('rating',index='movieId',columns='movieId')

# title 칼럼을 얻기 위해 movies와 합병
ratings_movies=pd.merge(ratings,movies,on='movieId')

#columns='title'로 title 칼럼으로 pivot 수행
ratings_matrix=ratings_movies.pivot_table('rating',index='movieId',columns='title')

In [71]:
P,Q=matrix_factorization(ratings_matrix.values,K=50,steps=200,learning_rate=0.01,r_lambda=0.01)
pred_matrix=np.dot(P,Q.T)

### iteration step : 0  rmse:  3.3750897479512445
### iteration step : 10  rmse:  3.3599635148116462
### iteration step : 20  rmse:  3.3353939286973278
### iteration step : 30  rmse:  3.2871196118332597
### iteration step : 40  rmse:  3.188960175697435
### iteration step : 50  rmse:  2.999934953055246
### iteration step : 60  rmse:  2.6842216825334426
### iteration step : 70  rmse:  2.2567331943484645
### iteration step : 80  rmse:  1.7896616636795306
### iteration step : 90  rmse:  1.36311369208508
### iteration step : 100  rmse:  1.0210300220038908
### iteration step : 110  rmse:  0.7669582241020713
### iteration step : 120  rmse:  0.5844966327516745
### iteration step : 130  rmse:  0.45397853509526637
### iteration step : 140  rmse:  0.3594576144952893
### iteration step : 150  rmse:  0.28987583246960613
### iteration step : 160  rmse:  0.2379319898852414
### iteration step : 170  rmse:  0.19868316494423963
### iteration step : 180  rmse:  0.16862496601086513
### iteration step : 19

- 예측 사용자-아이템 평점 행렬을 영화 타이틀을 col로 가지는 df로 변경하기

In [73]:
rating_pred_matrix=pd.DataFrame(data=pred_matrix,index=ratings_matrix.index,columns=ratings_matrix.columns)
rating_pred_matrix.head(3)

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...All the Marbles (1981),...And Justice for All (1979),00 Schneider - Jagd auf Nihil Baxter (1994),1-900 (06) (1994),10 (1979),10 Cent Pistol (2015),10 Cloverfield Lane (2016),10 Items or Less (2006),10 Things I Hate About You (1999),10 Years (2011),"10,000 BC (2008)",100 Girls (2000),100 Streets (2016),101 Dalmatians (1996),101 Dalmatians (One Hundred and One Dalmatians) (1961),101 Dalmatians II: Patch's London Adventure (2003),101 Reykjavik (101 Reykjavík) (2000),102 Dalmatians (2000),10th & Wolf (2006),"10th Kingdom, The (2000)","10th Victim, The (La decima vittima) (1965)","11'09""01 - September 11 (2002)",11:14 (2003),"11th Hour, The (2007)",12 Angry Men (1957),12 Angry Men (1997),12 Chairs (1971),12 Chairs (1976),12 Rounds (2009),12 Years a Slave (2013),...,Zathura (2005),Zatoichi and the Chest of Gold (Zatôichi senryô-kubi) (Zatôichi 6) (1964),Zazie dans le métro (1960),Zebraman (2004),"Zed & Two Noughts, A (1985)",Zeitgeist: Addendum (2008),Zeitgeist: Moving Forward (2011),Zeitgeist: The Movie (2007),Zelary (2003),Zelig (1983),Zero Dark Thirty (2012),Zero Effect (1998),"Zero Theorem, The (2013)",Zero de conduite (Zero for Conduct) (Zéro de conduite: Jeunes diables au collège) (1933),Zeus and Roxanne (1997),Zipper (2015),Zodiac (2007),Zombeavers (2014),Zombie (a.k.a. Zombie 2: The Dead Are Among Us) (Zombi 2) (1979),Zombie Strippers! (2008),Zombieland (2009),Zone 39 (1997),"Zone, The (La Zona) (2007)",Zookeeper (2011),Zoolander (2001),Zoolander 2 (2016),Zoom (2006),Zoom (2015),Zootopia (2016),Zulu (1964),Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
movieId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,-0.758233,-0.052356,-0.756763,-1.068458,0.374390,-0.347095,0.827393,-0.348838,0.247671,0.165586,0.282783,0.038266,0.290301,0.324597,0.307728,0.086166,0.563492,0.290817,0.312501,0.101077,0.358749,-0.640350,-0.370520,0.208891,0.713583,-0.146131,-0.490882,-0.055080,-1.296807,-0.176326,0.152926,0.105798,0.237644,-0.818375,-0.343055,-0.098192,0.210091,0.428078,0.112718,0.057621,...,-0.187265,0.521611,0.297461,0.258622,-0.501279,-0.442185,-0.602635,-0.143700,0.850990,0.165190,0.391401,0.936998,0.130533,-0.293678,-0.425053,0.042371,-0.583682,-0.677361,-0.447478,0.039424,-0.718930,-0.317645,-0.854337,0.609782,0.129936,-0.232532,-0.021407,-0.132475,0.984525,-0.246026,-0.083644,-0.280219,0.280893,-0.156348,0.324372,-0.386604,-0.145946,0.163610,0.461678,-0.184093
2,0.262478,-0.133071,-0.110659,-0.374049,0.441585,0.384656,0.082095,-0.342185,-0.832278,0.166675,0.495035,0.314001,0.131488,-0.179685,-0.053513,-0.040035,0.486862,-0.044437,-0.119887,0.134710,0.271811,0.810075,0.197983,0.460858,0.727426,0.293826,-0.047625,-0.255079,0.002916,0.276596,0.290672,0.142992,-0.137102,-0.067604,0.387627,0.162499,-1.043640,0.070358,0.363119,-0.185451,...,-0.308718,-1.309931,0.468260,0.157571,0.514745,1.127697,0.260313,0.156650,0.418566,-0.166040,0.327056,-0.274339,-0.258417,0.078386,-0.016262,-0.534788,-0.072680,0.469495,0.177813,-0.118914,0.293890,0.335335,-0.295754,-0.396453,-0.326640,0.513177,0.043611,0.702675,0.118769,-0.608559,-0.034726,0.476649,-0.534565,-0.631697,-0.044196,0.994124,-0.419517,-0.418244,-0.037347,-0.222577
3,-0.252080,-0.460726,0.082015,-0.547901,-0.177238,0.049936,0.103018,-0.651234,-0.200584,0.634052,0.418076,0.342257,0.267310,-0.708546,0.753267,-0.475858,0.550255,-0.106564,-0.556894,0.564272,-0.619994,0.528588,0.378427,-0.800294,-0.235684,0.112360,-0.115177,0.168395,-0.767172,0.568721,-0.670013,-0.114777,-0.205150,0.103062,-0.000577,0.282324,0

- 위의 예측 사용자-아이템 평점 행렬 정보로 개인화된 영화 추천 짆ㅇ

In [75]:
#사용자가 관람하지 않은 영화명 추출
unseen_list=get_unseen_movies(ratings_matrix,9)

#잠재 요인 협업 필터링으로 영화 추천
recomm_movies=recomm_movies = recomm_movie_by_userid(ratings_pred_matrix, 9, unseen_list, top_n=10)

#평점 데이터를 df로 생성
recomm_movies=pd.DataFrame(data=recomm_movies.values,index=recomm_movies.index,columns=['pred_score'])
recomm_movies

,pred_score
title,
Body of Evidence (1993),2.967085
Blind Date (1984),2.967085
"Last Seduction, The (1994)",1.837418
Sunshine State (2002),1.210886
Back to the Future Part III (1990),1.177924
Back to the Future Part II (1989),1.146718
"Lord of the Rings: The Fellowship of the Ring, The (2001)",1.118403
Back to the Future (1985),1.018926
"Producers, The (1968)",1.002248
